# CH10 数据分组/数据透视表（菜品分类）

In [1]:
import pandas as pd

## 10.1数据分组

数据分组是根据一个或者多个键将数据分为若干组，将数据分成若干组<br>
然后对分组后的数据进行汇总计算，最后将汇总计算后的结果进行合并<br>

### 10.1.1分组键是列名

In [2]:
data_dict = {"用户ID":[59224,55295,46035,2459,22179],
             "客户分类":["A类","B类","A类","C类","B类"],
             "区域":["一线城市","三线城市","二线城市","一线城市","三线城市"],
             "是否省会":["是","否","是","是","否",],
             "7月销量":[6,37,8,7,9],
             "8月销量":[20,27,1,8,12],
             "9月销量":[0,35,8,14,4]}
df = pd.DataFrame(data_dict)
df

,用户ID,客户分类,区域,是否省会,7月销量,8月销量,9月销量
0,59224,A类,一线城市,是,6,20,0
1,55295,B类,三线城市,否,37,27,35
2,46035,A类,二线城市,是,8,1,8
3,2459,C类,一线城市,是,7,8,14
4,22179,B类,三线城市,否,9,12,4


按照一列进行分组

In [3]:
df.groupby("客户分类")

返回的是DataFrameGroupBy对象，而不是一个DataFrame对象

In [4]:
df.groupby("客户分类").count()

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,2,2,2,2,2,2
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


In [5]:
df.groupby("客户分类").sum()

,用户ID,7月销量,8月销量,9月销量
客户分类,,,,
A类,105259,14,21,8
B类,77474,46,39,39
C类,2459,7,8,14


In [6]:
df.groupby(["客户分类","区域"]).count()

用户ID  是否省会  7月销量  8月销量  9月销量
客户分类 区域                                
A类   一线城市     1     1     1     1     1
     二线城市     1     1     1     1     1
B类   三线城市     2     2     2     2     2
C类   一线城市     1     1     1     1     1

In [7]:
df.groupby(["客户分类","区域"]).sum()

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  46035     8     1     8
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

不过也仅对某一列进行分组计算

In [8]:
df.groupby("客户分类")["用户ID"].count()

客户分类
A类    2
B类    2
C类    1
Name: 用户ID, dtype: int64

### 10.1.2分组键是Series

将DataFrame中的某一列取出来就是一个Series

In [9]:
df["客户分类"]

0    A类
1    B类
2    A类
3    C类
4    B类
Name: 客户分类, dtype: object

给groupby()方法传入了什么，其他都一样，可以按照一个或多个Series进行分组

In [10]:
df.groupby(df["客户分类"]).count()

,用户ID,区域,是否省会,7月销量,8月销量,9月销量
客户分类,,,,,,
A类,2,2,2,2,2,2
B类,2,2,2,2,2,2
C类,1,1,1,1,1,1


In [11]:
df.groupby([df["客户分类"],df["区域"]]).sum()

用户ID  7月销量  8月销量  9月销量
客户分类 区域                           
A类   一线城市  59224     6    20     0
     二线城市  46035     8     1     8
B类   三线城市  77474    46    39    39
C类   一线城市   2459     7     8    14

In [12]:
df.groupby(df["客户分类"])["用户ID"].count()

客户分类
A类    2
B类    2
C类    1
Name: 用户ID, dtype: int64

### 10.1.3Aggregate方法

In [13]:
data_dict = {"用户ID":[59224,55295,46035,2459,22179],
             "客户分类":["A类","B类","A类","C类","B类"],
             "7月销量":[6,37,8,7,9],
             "8月销量":[20,27,1,8,12]}
df = pd.DataFrame(data_dict)
df

,用户ID,客户分类,7月销量,8月销量
0,59224,A类,6,20
1,55295,B类,37,27
2,46035,A类,8,1
3,2459,C类,7,8
4,22179,B类,9,12


Agggreate方法之所以神奇，是因为有如下两大特点<br>
1.一次可以使用多种汇总方式<br>
2.可以针对不同列进行不同的汇总运算<br>

In [14]:
df.groupby("客户分类").aggregate(["count","sum"])

用户ID          7月销量      8月销量    
     count     sum count sum count sum
客户分类                                  
A类       2  105259     2  14     2  21
B类       2   77474     2  46     2  39
C类       1    2459     1   7     1   8

In [15]:
df.groupby("客户分类").aggregate({"用户ID":"count","7月销量":["sum","count"],"8月销量":"sum"})

用户ID 7月销量       8月销量
     count  sum count  sum
客户分类                      
A类       2   14     2   21
B类       2   46     2   39
C类       1    7     1    8

### 10.1.4对分组后的结果重置索引

由于DataFrameGroupBy对象经过汇总运算后得到的形式不是标准的DataFrame形式，所以需要对分组结果进行转化<br>
具体使用reset_index()方法

In [16]:
df.groupby("客户分类").sum()

,用户ID,7月销量,8月销量
客户分类,,,
A类,105259,14,21
B类,77474,46,39
C类,2459,7,8


In [17]:
df.groupby("客户分类").sum().reset_index()

,客户分类,用户ID,7月销量,8月销量
0,A类,105259,14,21
1,B类,77474,46,39
2,C类,2459,7,8


## 10.2数据透视表

数据透视表与分组不同之处在于，透视表是在行与列的方向上同时拆分<br>
无论是Excel还是Python，这都是相当常见的作法，务必要牢记<br>
pd.pivot_table(data, values(值) = , index(行) = , columns(列) = , aggfunc(计算类型) = , fill_value(对NA的填充) = ,<br>
margins = , dropna(是否删除缺失值) = True , margins_name(合计列名) = )

In [18]:
data_dict = {"用户ID":[59224,55295,46035,2459,22179],
             "客户分类":["A类","B类","A类","C类","B类"],
             "区域":["一线城市","三线城市","二线城市","一线城市","三线城市"],
             "是否省会":["是","否","是","是","否",],
             "7月销量":[6,37,8,7,9],
             "8月销量":[20,27,1,8,12],
             "9月销量":[0,35,8,14,4]}
df = pd.DataFrame(data_dict)
df

,用户ID,客户分类,区域,是否省会,7月销量,8月销量,9月销量
0,59224,A类,一线城市,是,6,20,0
1,55295,B类,三线城市,否,37,27,35
2,46035,A类,二线城市,是,8,1,8
3,2459,C类,一线城市,是,7,8,14
4,22179,B类,三线城市,否,9,12,4


In [19]:
pd.pivot_table(df, values = "用户ID", columns = "区域", index = "客户分类", aggfunc = "count")

区域,一线城市,三线城市,二线城市
客户分类,,,
A类,1.0,NaN,1.0
B类,NaN,2.0,NaN
C类,1.0,NaN,NaN


因为Margin默认是为False的，所以如果想要合计的值，必须再手动改，默认列明为All，也可以手动再用margin_name去调整

In [20]:
pd.pivot_table(df, values = "用户ID", columns = "区域", index = "客户分类", aggfunc = "count",
               margins = True, margins_name = "合计")

区域,一线城市,三线城市,二线城市,合计
客户分类,,,,
A类,1.0,NaN,1.0,2
B类,NaN,2.0,NaN,2
C类,1.0,NaN,NaN,1
合计,2.0,2.0,1.0,5


通过fill_value参数，将空缺值填为0

In [21]:
pd.pivot_table(df, values = "用户ID", columns = "区域", index = "客户分类", aggfunc = "count",
               margins = True, fill_value = 0)

区域,一线城市,三线城市,二线城市,All
客户分类,,,,
A类,1,0,1,2
B类,0,2,0,2
C类,1,0,0,1
All,2,2,1,5


如果对不同的列有不同的计算方式，需要用字典的方式传入

In [24]:
pd.pivot_table(df, values=["用户ID", "7月销量"], columns = "区域",
               index = "客户分类", aggfunc ={ "用户ID": "count", "7月销量": "sum" })

7月销量            用户ID          
区域   一线城市  三线城市 二线城市 一线城市 三线城市 二线城市
客户分类                               
A类    6.0   NaN  8.0  1.0  NaN  1.0
B类    NaN  46.0  NaN  NaN  2.0  NaN
C类    7.0   NaN  NaN  1.0  NaN  NaN

最后再用reset_index()方法对数据进行DataFrame化

In [25]:
pd.pivot_table(df, values = "用户ID", columns = "区域", index = "客户分类", aggfunc = "count").reset_index()

区域,客户分类,一线城市,三线城市,二线城市
0,A类,1.0,NaN,1.0
1,B类,NaN,2.0,NaN
2,C类,1.0,NaN,NaN
